<a href="https://colab.research.google.com/github/DmitryKutsev/DeepHW/blob/master/hw3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install youtokentome

     |████████████████████████████████| 1.7MB 12.2MB/s 


In [4]:
!wget https://developer.nvidia.com/compute/cuda/9.2/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64 -O cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb
!dpkg -i cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb
!apt-key add /var/cuda-repo-9-2-local/7fa2af80.pub
!apt-get update
!apt-get install cuda

--2020-10-28 21:09:13--  https://developer.nvidia.com/compute/cuda/9.2/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64
Resolving developer.nvidia.com (developer.nvidia.com)... 152.199.0.24
Connecting to developer.nvidia.com (developer.nvidia.com)|152.199.0.24|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://developer.download.nvidia.com/compute/cuda/9.2/secure/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb?vjM2U8NalSQ-6LMguIt1vob3YTdtBNhFAl8B8qnHCgZc5PYVRsYMV-0lhsMxhwtJR2qYlyFtYtGPSL7bkLzlK4gwA8LRB5rNRHcIfjLLbzITo_jtjgJdq-Z5Hc2W7U7XGFGB9pQHkIRi1Z_RwgPPLfzb_xUXYbdpGIBdda2_3t_TXAmdoM1WLHOKapuq2R7_UgMXlEoXRTL8UKprWuU [following]
--2020-10-28 21:09:13--  https://developer.download.nvidia.com/compute/cuda/9.2/secure/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb?vjM2U8NalSQ-6LMguIt1vob3YTdtBNhFAl8B8qnHCgZc5PYVRsYMV-0lhsMxhwtJR2qYlyFtYtGPSL7bkLzlK4gwA8LRB5rNRHcIfjLLbzITo_jtjgJdq-Z5Hc2W

In [ ]:
!pip install mxnet-cu92

     |█████████████████████████▌      | 628.2MB 1.3MB/s eta 0:02:01

In [ ]:
print(torch.cuda.device_count())

In [ ]:
!pip install --upgrade --force-reinstall -q http://download.pytorch.org/whl/{accelerator}/torch-0.4.0-{platform}-linux_x86_64.whl torchvision

In [2]:
import math
import numpy as np
import pandas as pd

import random
import json
import torch
from torch.utils.data import Dataset, DataLoader

from tqdm import tqdm

from matplotlib import pyplot as plt

import youtokentome as yttm

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
!unzip '/content/drive/My Drive/unsupervised.csv.zip'

Archive:  /content/drive/My Drive/unsupervised.csv.zip
  inflating: unsupervised.csv        


In [5]:
!ls

drive  sample_data  unsupervised.csv


In [6]:
questions = pd.read_csv('unsupervised.csv')
questions.question = questions.question.map(lambda x: x.lower())

In [7]:
# для "обучения" bpe модели нам нужно сохранить данные для обучения в отдельный файл
# где будут построчно храниться тексты
# раскомментируйте этот код, чтобы собрать такой файл
with open('for_bpe.txt', 'w', encoding='utf-8') as f:
    for que in questions.question:
        f.write(que + '\n')
        
# параметры
vocab_size = 30_000
model_path = 'pretrained_bpe_lm.model'

In [8]:
%%time
# обучаем
# раскомментируйте этот код, чтобы обучить bpe
yttm.BPE.train(data='for_bpe.txt', vocab_size=vocab_size, model=model_path)

CPU times: user 25.3 s, sys: 2.54 s, total: 27.8 s
Wall time: 16.8 s


In [9]:
# загружаем токенизатор
tokenizer = yttm.BPE(model=model_path)

In [10]:
tokenized = []

batch_size = 256

for i_batch in tqdm(range(math.ceil(len(questions.question) / batch_size))):
    
    tokenized.extend(tokenizer.encode(
        list(questions.question[i_batch*batch_size:(i_batch+1)*batch_size]), bos=True, eos=True))

100%|██████████| 9905/9905 [00:33<00:00, 291.61it/s]


In [11]:
class LanguageModelData(torch.utils.data.Dataset):
    
    def __init__(self, data, max_len, pad_index, eos_index):
        
        self.data = data
        
        self.max_len = max_len
        
        self.pad_index = pad_index
        self.eos_index = eos_index
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index):
        
        sequence = self.data[index][:self.max_len]
        
        # исходная последовательность
        x = sequence[:]
        # нужно предсказать смещенную последовательность
        y = sequence[1:] + [self.eos_index]
        
        assert len(x) == len(y)
        
        pads = [self.pad_index] * (self.max_len - len(x))
        
        x = torch.tensor(x + pads).long()
        y = torch.tensor(y + pads).long()
        
        return x, y

In [15]:
batch_size = 64

max_len = 32

pad_index = 0
eos_index = 3

In [16]:
# надо выбрать максимальную длину
lengths = np.array([len(x) for x in tokenized])
np.percentile(lengths, q=95)

28.0

In [17]:
random.shuffle(tokenized)

validation_start_index = int(len(tokenized) * 0.05)

In [18]:
train_dataset = LanguageModelData(data=tokenized[:-validation_start_index], max_len=max_len, 
                                  pad_index=pad_index, eos_index=eos_index)
validation_dataset = LanguageModelData(data=tokenized[-validation_start_index:], max_len=max_len,
                                       pad_index=pad_index, eos_index=eos_index)

len(train_dataset), len(validation_dataset)

(2408671, 126772)

In [19]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size)
validation_loader = torch.utils.data.DataLoader(validation_dataset, batch_size=batch_size)

In [20]:

for x, y in train_loader:
    break

In [21]:
x

tensor([[   2, 2009, 1833,  ...,    0,    0,    0],
        [   2, 2664, 2324,  ...,    0,    0,    0],
        [   2, 1991, 2193,  ...,    0,    0,    0],
        ...,
        [   2, 1904, 2008,  ...,    0,    0,    0],
        [   2, 3109, 1827,  ...,    0,    0,    0],
        [   2, 1991, 1966,  ...,    0,    0,    0]])

In [22]:
progress_bar = tqdm(total=len(validation_loader.dataset), desc='Testing')

for x, y in validation_loader:
    progress_bar.update(x.size(0))
    
progress_bar.close()

Testing: 100%|██████████| 126772/126772 [00:02<00:00, 59732.16it/s]


In [23]:
class LanguageModel(torch.nn.Module):
    
    def __init__(self, vocab_size, embedding_dim, model_dim, num_layers, dropout, padding_idx):
        
        super().__init__()
        
        # просто эмбеддинги
        self.embedding_layer = torch.nn.Embedding(num_embeddings=vocab_size, 
                                                  embedding_dim=embedding_dim,
                                                  padding_idx=padding_idx)
        
        # просто lstm
        self.lstm = torch.nn.LSTM(input_size=embedding_dim, 
                                  hidden_size=model_dim,
                                  num_layers=num_layers, 
                                  dropout=0.3,
                                  batch_first=True)
        
        # выходная матрица эмбеддингов
        # количество выходных фичей равно размеру словаря
        # то есть это задача мультиклассовой классификации, но только классов очень много
        self.language_model_head = torch.nn.Linear(in_features=model_dim,
                                                   out_features=vocab_size,
                                                   bias=False)
        
    def forward(self, x):
        
        x = self.embedding_layer(x)
        
        x, _ = self.lstm(x)
        
        # к каждому элементу последовательности применяется выходная матрица эмбеддингов, 
        # которая переводит вектор lstm в предсказание конкретного слова
        x = self.language_model_head(x)
        
        return x

In [3]:
torch.cuda.is_available(), 'у вас не находится гпу'

(True, 'у вас не находится гпу')

In [31]:
device = torch.device('cuda:1')

In [ ]:
embedding_dim = 128
model_dim = 128
num_layers = 2
dropout = 0.35

In [ ]:
model = LanguageModel(vocab_size=vocab_size, embedding_dim=embedding_dim,
                      model_dim=model_dim, num_layers=num_layers,
                      dropout=dropout, padding_idx=pad_index)
model.to(device)

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [ ]:
print(f'Количество обучаемых параметров в сети: {count_parameters(model):,}')

In [ ]:
x = x.to(device)
y = y.to(device)